In [1]:
import torch
import peewee
import pandas as pd
import torch.optim as optim
import torch.nn as nn

In [2]:
import os, sys

root_dir = os.path.dirname("../")
print("Root Directory :", root_dir)

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print("Path added")
else:
    print("Path exists")

Root Directory : ..
Path added


In [3]:
from src.model.data import create_dataloaders
from src.model.main import LSTM

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

#### Prepare Data

In [5]:
db = peewee.SqliteDatabase("../data/dataset.sqlite3")
conn = db.connection()
tables = db.get_tables()

In [6]:
stock = "AAPL"
stock_df = pd.read_sql(f"SELECT * FROM {tables[0]} WHERE symbol = '{stock}'", conn)
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338142 entries, 0 to 338141
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   symbol     338142 non-null  object 
 1   timestamp  338142 non-null  object 
 2   open       338142 non-null  float64
 3   high       338142 non-null  float64
 4   low        338142 non-null  float64
 5   close      338142 non-null  float64
 6   volume     338142 non-null  float64
dtypes: float64(5), object(2)
memory usage: 18.1+ MB


In [7]:
db.close()

True

#### Hyperparameters

In [8]:
input_size = 5  # OHLCV
hidden_size = 50
num_layers = 2
output_size = 5  # Predicting OHLCV

seq_length = 10
batch_size = 32
test_size = 0.2
num_epochs = 10
learning_rate = 0.001

In [9]:
train_dataloader, test_dataloader = create_dataloaders(stock_df, seq_length, batch_size, test_size)

Cols ['open', 'high', 'low', 'close', 'volume']


In [10]:
model = LSTM(input_size, hidden_size, num_layers, output_size, device)
model.to(device)
print("LSTM on device :", model.device)

LSTM on device : cuda:0


In [11]:
device = model.device
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.float().to(device), targets.float().to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")

print("Training finished.")

Epoch [1/10], Loss: 0.0001
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Training finished.


#### Saving The Model

In [12]:
model.eval()

print("Saving model...")
torch.save(model, "../model/lstm_model.pth")
print("Model saved.")

Saving model...
Model saved.
